# Langchain Tools

In [11]:
import { z } from "zod";

const stringSchema = z.string();
stringSchema.parse("Hello, Zod!");

"Hello, Zod!"

In [12]:
// stringSchema.parse(2323); // typo checking faild

In [13]:
// define a schema for an object

// 基础类型
const stringSchema = z.string();
const numberSchema = z.number();
const booleanSchema = z.boolean();

// 数组
const stringArraySchema = z.array(z.string());
stringArraySchema.parse(["apple", "banana", "cherry"]); 

// 对象
const personSchema = z.object({
  name: z.string(),
  age: z.number(),
  // 可选类型
  isStudent: z.boolean().optional(),
  // 默认值
  home: z.string().default("no home")
});

// 联合类型
const mixedTypeSchema = z.union([z.string(), z.number()]);
mixedTypeSchema.parse("hello"); 
mixedTypeSchema.parse(42); 

42

In [14]:
const getCurrentWeatherSchema = z.object({
    location: z.string().describe("The city and state, e.g. San Francisco, CA"),
    unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
});

In [15]:
import { zodToJsonSchema } from "zod-to-json-schema";

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema)

console.log(paramSchema);

{
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [16]:
// JSON Schema for getCurrentWeatherSchema

import { load } from "dotenv";
import OpenAI from "openai";

const env = await load();

const openai = new OpenAI({
    apiKey: env["AZURE_OPENAI_API_KEY"],
    baseURL: `https://${env["AZURE_OPENAI_API_INSTANCE_NAME"]}.openai.azure.com/openai/deployments/${env["AZURE_OPENAI_API_DEPLOYMENT_NAME"]}`,
    defaultQuery: { 'api-version':  env["AZURE_OPENAI_API_VERSION"] },
    defaultHeaders: { 'api-key': env["AZURE_OPENAI_API_KEY"] },
});

In [17]:
import { ChatOpenAI } from '@langchain/openai'

const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema),
            }
        }
    ]
})

await modelWithTools.invoke("北京的天气怎么样");

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          function: [Object],
          id: "call_v2NwQgM03MQ2q3SA3RN960d1",
          type: "function"
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        function: {
          arguments: '{\n  "location": "Beijing",\n  "unit": "celsius"\n}',
          name: "getCurrentWeather"
        },
        id: "call_v2NwQgM03MQ2q3SA3RN960d1",
        type: "function"
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 24, promptTokens: 88, totalTokens: 112 },
    finish_reason: "tool_calls"
  }
}

In [18]:
const getCurrentTimeSchema = z.object({
    format: z
      .enum(["iso", "locale", "string"])
      .optional()
      .describe("The format of the time, e.g. iso, locale, string"),
  });
  
  zodToJsonSchema(getCurrentTimeSchema)

{
  type: "object",
  properties: {
    format: {
      type: "string",
      enum: [ "iso", "locale", "string" ],
      description: "The format of the time, e.g. iso, locale, string"
    }
  },
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [19]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithMultiTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema)
            }
        },
        {
            type: "function",
            function: {
                name: "getCurrentTime",
                description: "Get the current time in a given format",
                parameters: zodToJsonSchema(getCurrentTimeSchema)
            }
        }
    ]
})

## Control model to call function tools

### Tools choices

```ts
model.bind({
    tools: [
        ...
    ],
    tool_choice: "none"
})

```

### Tools specification


```ts
const modelWithForce = model.bind({
    tools: [
        ...
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    }
})
```

## Use tools to mark data

In [20]:
const taggingSchema = z.object({
    emotion:z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
    language: z.string().describe("文本的核心语言（应为ISO 639-1代码）"),
  });

In [21]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelTagging = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "tagging",
                description: "为特定的文本片段打上标签",
                parameters: zodToJsonSchema(taggingSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "tagging"
        }
    }
})

In [22]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";
// import ChatPromptTemplate
import { ChatPromptTemplate } from '@langchain/core/prompts'

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser())

In [23]:
await chain.invoke({
    input: "hello world"
})

// [ { type: "tagging", args: { emotion: "neutral", language: "en" } } ]

await chain.invoke({
    input: "写代码太难了，👴 不干了"
})
// [ { type: "tagging", args: { emotion: "neg", language: "zh" } } ]

await chain.invoke({
    // 日语，圣诞快乐
    input: "メリークリスマス!"
})
// [ { type: "tagging", args: { emotion: "pos", language: "ja" } } ]

await chain.invoke({
    input: "我非常喜欢 AI，特别是 LLM，因为它非常 powerful"
})
// [ { type: "tagging", args: { emotion: "pos", language: "zh" } } ]

[ { type: "tagging", args: { emotion: "pos", language: "en" } } ]

## Use tools to extract information data

In [24]:
const personExtractionSchema = z.object({
    name: z.string().describe("人的名字"),
    age: z.number().optional().describe("人的年龄")
}).describe("提取关于一个人的信息");

In [25]:
const relationExtractSchema = z.object({
    people: z.array(personExtractionSchema).describe("提取所有人"),
    relation: z.string().describe("人之间的关系, 尽量简洁")
})

In [26]:
const schema = zodToJsonSchema(relationExtractSchema)

console.log(schema)

{
  type: "object",
  properties: {
    people: {
      type: "array",
      items: {
        type: "object",
        properties: { name: [Object], age: [Object] },
        required: [ "name" ],
        additionalProperties: false,
        description: "提取关于一个人的信息"
      },
      description: "提取所有人"
    },
    relation: { type: "string", description: "人之间的关系, 尽量简洁" }
  },
  required: [ "people", "relation" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [27]:
const model = new ChatOpenAI({
    temperature: 0 
})

const modelExtract = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "relationExtract",
                description: "提取数据中人的信息和人的关系",
                parameters: zodToJsonSchema(relationExtractSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "relationExtract"
        }
    }
})

// Here, the prompt design, we use "Think carefully, you have enough time to think rigorously" to enhance the quality of the llm output, and then use "If not provided clearly, please do not guess, you can only extract part of the information" to reduce the fantasy problem of llm.
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考，你有充足的时间进行严谨的思考，然后提取文中的相关信息，如果没有明确提供，请不要猜测，可以仅提取部分信息"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelExtract).pipe(new JsonOutputToolsParser())

In [30]:
await chain.invoke({
    input: "小明现在 18 岁了，她妈妈是小丽"
})

[
  {
    type: "relationExtract",
    args: { people: [ { name: "小明", age: 18 } ], relation: "小明的妈妈是小丽" }
  }
]

In [31]:
await chain.invoke({
    input: "我是小明现在 18 岁了，我和小 A、小 B 是好朋友，都一样大"
})

[
  {
    type: "relationExtract",
    args: {
      people: [
        { name: "小明", age: 18 },
        { name: "小A", age: 18 },
        { name: "小B", age: 18 }
      ],
      relation: "好朋友"
    }
  }
]

In [32]:
await chain.invoke({
    input: "我是小明"
})

[
  {
    type: "relationExtract",
    args: { people: [ { name: "小明", age: 20 } ], relation: "" }
  }
]